In [21]:
import os
import csv
import math
import librosa
import librosa.display
import numpy as np
import pandas as pd
from IPython.display import Audio
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from python_speech_features import mfcc
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import kurtosis, skew

In [2]:
# load training data.
rootLocation = "data/genres" # directory which contains the folder of genres. 
genres = os.listdir(rootLocation)
genres.remove('.DS_Store') if '.DS_Store' in genres else None
data = list() # read .au files from librosa library
labels = list() # list of labels

for genre in genres:
    clips = os.listdir(rootLocation+"/{}".format(genre))
    clips.remove('.DS_Store') if '.DS_Store' in clips else None
    
    for clip in clips: # using first 2 songs from each genre. remove [:2] to loop through all songs.
        musicLabel = clip.split('.',1)[0]
        musicData = librosa.load('data/genres/{}/{}'.format(genre,clip))
        #musicData = rootLocation + "/{}/{}".format(genre,clip)
        
        labels.append(musicLabel)
        data.append(musicData)
        

In [3]:
# Load Validation data:
v_Location = "data/validation" # directory which contains the folder of genres. 
v_clips = os.listdir(v_Location)
v_clips.array('.DS_Store') if '.DS_Store' in v_clips else None
v_data = list() # read .au files from librosa library  
v_label = list()
for v_clip in v_clips:
    v_musicData = librosa.core.load(v_Location+'/{}'.format(v_clip))
    #musicData = rootLocation+"/{}/{}".format(genre,clip)
    v_label.append(v_clip)
    v_data.append(v_musicData)

In [ ]:
# plot spectrogram from the data read
c = np.abs(librosa.stft(data[40][0]))
librosa.display.specshow(librosa.power_to_db(c**2, ref=np.max),sr=data[2][1], y_axis='log', x_axis='time')
plt.colorbar()
plt.show()

In [ ]:
# play the selected audio
Audio(data = v_data[71][0], rate = v_data[71][1])

In [ ]:
# First Feature ex
# fast Fourier Transform feature extraction
def fourierfeatures(data, n = 1000): # data in format [[y], sample_rate]
    fft_features= []
    for clip in data:
        features = abs(fft(clip[0]))[:n]
        m = max(features)
        norm_features = [x / m for x in features]
        fft_features.append(norm_features) # normalize the numbers between 0 and 1.
    return fft_features

In [ ]:
# MFCC feature extraction
def mfccfeatures(data): # data in format [[y], sample_rate]
    mfcc_features= []
    for clip in data:
        features= mfcc(clip[0])
        avg_f = np.mean(features, axis=0)
        m = max(avg_f)
        norm_features = [x / m for x in avg_f]
        mfcc_features.append(norm_features) 
    return mfcc_features

In [ ]:
# Custom Feature extraction + MFCC feature extraction
def customfeatures(data): # data in format [[y], sample_rate]
    mfcc_features= []
    for clip in data:
        ## mfcc features as the base. 
        features= mfcc(clip[0],samplerate = clip[1], nftt = 600)
        avg_f = np.mean(features, axis=0)
        
        ## add tempo value as the 14th feature
        tempo = [a/15 for a in librosa.beat.tempo(y=clip[0], sr=clip[1])]
        avg_f= np.append(avg_f, tempo)  # add tempo as 14th featur
        
        ## normalize between 0 and 1
        #m = max(abs(avg_f))
        #norm_features = [x/m for x in avg_f]
        mfcc_features.append(avg_f) 
    return mfcc_features

In [33]:
def getBest_Features(data):
    i = 0
    best_features= list()
    for clip in data:
        features= mfcc(clip[0])
        avg_f = np.mean(features, axis=0)
        std_f = np.std(features,axis=0)
        skew_f = skew (features, axis =0)
        kurt_f = kurtosis(features, axis =0)
        addList = np.concatenate((avg_f,std_f,skew_f,kurt_f))
        best_features.append(addList)
        i +=1
        print(i)
    return best_features

In [ ]:
rhythm= librosa.feature.tempogram(data[0][0], sr = data[0][1])
avg_f = np.mean(rhythm, axis=0)

#print(pd.DataFrame(avg_f))



In [34]:
training_features = getBest_Features(data = data)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [35]:
validating_features = getBest_Features(data = v_data)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [37]:
print(pd.DataFrame(training_features))

           0          1          2          3          4          5   \
0   -1.840303   0.845768 -15.359319   4.642455  -9.664028   7.107428   
1   -3.610452   2.443410  -3.376578   6.012648  -8.670714  12.252719   
2   -1.282971   6.213523 -17.201779   0.652490 -15.824363   9.933808   
3   -3.715396  10.511946  -4.284404   6.107004  -9.448897  10.236501   
4   -2.317288   4.293837 -21.366419   1.427945 -28.434836   1.840336   
5   -2.478954   5.184075 -14.052245   7.738119 -23.033810   5.836537   
6   -2.469019  14.230047 -21.393767   4.203986   0.249940   6.615951   
7   -2.587453  10.331691 -17.111416   9.647508  -4.898608  14.712234   
8   -1.887506   0.257359 -12.462396   8.435288  -7.363134  12.383098   
9   -3.407196   1.970374 -12.368604  10.241314 -22.043232  -2.200615   
10  -2.523939  12.662200 -16.656316   1.057163 -24.187204  -1.626785   
11  -5.339363   6.575031  -5.755991  14.127932  -9.455431  14.664079   
12  -2.787255   7.664898 -16.903211  17.312907  -4.237497  -3.01

In [38]:
#COnvert to Pandas dataframe.. better compatibility with the algorithms
tdf = pd.DataFrame(training_features)
vdf = pd.DataFrame(validating_features)
ldf = pd.DataFrame(labels)

In [44]:
# encapsulate other ML libraries
def genericML1(trainingdata,traininglabels):
    model = RandomForestClassifier(n_estimators= 800) # constructor for ML class
    model.fit(trainingdata, traininglabels)
    return model

In [ ]:
# encapsulate other ML libraries
def genericML2(trainingdata,traininglabels):
    model = MLPClassifier(max_iter = 1000, random_state =2) # constructor for Ml class
    model.fit(trainingdata,traininglabels)
    return model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(tdf, ldf, test_size=0.2, random_state=32)

In [45]:
mlObj = genericML1(tdf,ldf.values.ravel())

In [46]:
prediction = mlObj.predict(vdf)

In [15]:
# make prediction
prediction = mlObj.predict(vdf)
#actual = list(y_test.values)
#print(list(actual.values))
count=0
for p,a in zip(prediction,actual):
    if p == a: count += 1 
print(count/len(prediction))
#print(y_test)
    
#print(v_label)

NameError: name 'actual' is not defined

In [47]:
# write to csv file
with open('predict.csv', 'w') as csvfile:
    writer = csv.writer(csvfile,lineterminator='\n')
    writer.writerow(['id', 'class'])
    for file , p_label in zip(v_label, prediction):
        writer.writerow([str(file),str(p_label)])
    

In [48]:
print(prediction)

['hiphop' 'reggae' 'jazz' 'country' 'country' 'jazz' 'blues' 'country'
 'classical' 'metal' 'jazz' 'classical' 'blues' 'blues' 'reggae' 'blues'
 'jazz' 'metal' 'blues' 'jazz' 'blues' 'disco' 'pop' 'pop' 'disco' 'metal'
 'blues' 'rock' 'hiphop' 'hiphop' 'metal' 'country' 'metal' 'jazz'
 'hiphop' 'metal' 'metal' 'country' 'country' 'classical' 'metal' 'metal'
 'jazz' 'country' 'classical' 'rock' 'disco' 'classical' 'reggae' 'reggae'
 'pop' 'metal' 'jazz' 'classical' 'pop' 'pop' 'metal' 'metal' 'metal'
 'pop' 'rock' 'pop' 'disco' 'rock' 'pop' 'blues' 'jazz' 'country' 'metal'
 'hiphop' 'pop' 'hiphop' 'classical' 'blues' 'country' 'reggae' 'metal'
 'jazz' 'metal' 'reggae' 'disco' 'jazz' 'jazz' 'metal' 'metal' 'rock'
 'pop' 'jazz' 'country' 'jazz' 'reggae' 'disco' 'jazz' 'country' 'jazz'
 'metal' 'reggae' 'reggae' 'blues' 'metal']
